In [1]:
import MeCab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.datasets import load_files

import lib.preprocesser as pre

In [34]:
import importlib
importlib.reload(pre)

<module 'lib.preprocesser' from '/Users/takizawatakafumi/code/ml_text_similarity/code/lib/preprocesser.py'>

In [10]:
dirlist = ["dokujo-tsushin","it-life-hack","kaden-channel","livedoor-homme",
           "movie-enter","peachy","smax","sports-watch","topic-news"]
color_list = ['red','orange','yellow','lime','green','cyan','navy','purple','pink','w','black']

In [11]:
#データ読み込み
df = pd.DataFrame(columns=["class","news"])

for i in tqdm(dirlist):
    path = "data/text/"+i+"/*.txt"
    files = glob.glob(path)
    files.pop()
    for j in tqdm(files):
        f = open(j)
        data = f.read() 
        f.close()
        t = pd.Series([i,"".join(data.split("\n")[3:])],index = df.columns)
        df  = df.append(t,ignore_index=True)

/Users/takizawatakafumi/anaconda/lib/python3.5/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [12]:
parser = pre.Parser()
#df['word_list'] = df['news'].map(parser.parse_text)

In [13]:
from gensim.models import word2vec
model_name = "./model/200features_20minwords_10context_len2alldata.model"
model = word2vec.Word2Vec.load(model_name)

In [14]:
text_vectornizer = pre.Text_Vectornizer(model=model,matrix_size=500,parser=parser)
matrix = text_vectornizer.text2matrix(df['news'].values[0])
#matrix = matrix.reshape(model.vector_size,-1)
print(matrix.shape)

(200, 500)


In [15]:
#df['text_matrix'] = df['news'].map(text_vectornizer.text2matrix)
text_matrix = np.array([text_vectornizer.text2matrix(text) for text in df['news'].values])

#print(text_matrix)

In [16]:
class_category, category_list = df['class'].factorize()
class_category[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
category_one_hot = encoder.fit_transform(class_category.reshape(-1,1))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_matrix, category_one_hot, test_size=0.33, random_state=42)

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import keras.models as models

In [22]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('lstm_model')
early = EarlyStopping(monitor="val_categorical_crossentropy", 
                      mode="max", 
                      patience=5) # probably needs to be more patient, but kaggle time is limited

checkpoint = ModelCheckpoint(weight_path, monitor='val_categorical_crossentropy', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

callbacks_list = [checkpoint, early]

In [23]:
# モデル構築

# 1つの学習データのStep数(今回は25)
length_of_sequence = 500
in_out_neurons = len(category_list)
n_hidden = 300

model = Sequential()
model.add(LSTM(n_hidden, batch_input_shape=(None, 200, length_of_sequence), return_sequences=False))
model.add(Dense(in_out_neurons))
model.add(Dropout(0.5))
model.add(Activation("softmax"))
#optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_accuracy'])
#model.compile(loss="mean_squared_error", optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 300)               961200    
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 2709      
_________________________________________________________________
dropout_1 (Dropout)          (None, 9)                 0         
_________________________________________________________________
activation_1 (Activation)    (None, 9)                 0         
Total params: 963,909
Trainable params: 963,909
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, #df['text_matrix'].values, 
                  y_train, 
                  batch_size=64, 
                  epochs=20,
                  validation_split=0.1)

Train on 4436 samples, validate on 493 samples
Epoch 1/20
2304/4436 [==============>...............] - ETA: 1:14 - loss: 2.1132 - categorical_accuracy: 0.2192

In [35]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.2965449935163476
Test accuracy: 0.9445130392669797


In [27]:
#model.save_weights('param.hdf5')
model_json_str = model.to_json()
open('lstm_model.json', 'w').write(model_json_str)
model.save_weights('lstm_weights.hdf5')

In [ ]:
matrix = [v.tolist() for v in m]
print(type(matrix))

In [24]:
predict = model.predict_classes(text_matrix)

In [25]:
print(predict)

[0 0 0 ... 8 2 7]


In [37]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [9]:
#model = models.model_from_json('lstm_model.json')
    #'lstm_weights.hdf5')
#model.summary()
import json
data = json.loads('lstm_model.json')
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)